In [2]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [1]:
import tensorflow 

import pandas as pd
import numpy as np
import os
import keras
import random
import cv2
import math
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Convolution2D, BatchNormalization
from tensorflow.keras.layers import Flatten, MaxPooling2D,Dropout

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

import warnings
warnings.filterwarnings("ignore")

In [ ]:
base_model = DenseNet121(weights='imagenet', include_top = False, input_shape = (128, 128, 3)) # förtränad modell "imagenet", från keras. Man skulle kunna träna sin egen från grunden istället. Tar typ 2 veckor med ett jättebra grafikkort (! ; def train(self):

x = base_model.output
x = GlobalAveragePooling3D()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(5,activation='softmax')(x)

In [ ]:

model=Model(inputs=base_model.input,outputs=predictions)
len(model.layers)

435

In [ ]:
for layer in model.layers[:-8]:
  layer.trainable = False

for layer in model.layers[-8:]:
  layer.trainable = True 

In [ ]:
model.layers[-8:]

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
# Använd för att läsa in random bilder och random kategorier

nr_images_per_style_random = 120
nr_of_styles_random = 10 # Glöm inte att ändra högst uppe vid dense, så att det är samma siffra där, å kör alla block ovanför

random.seed(42)
root = "/content/gdrive/MyDrive/project-art/datasets/Wikiart-dataset/wikiart/"
all_styles = os.listdir(root)
sample_styles =  random.sample(all_styles, nr_of_styles_random)
images_labels = []

for folder_name in sample_styles:
  random_images = random.sample(os.listdir(os.path.join(root, folder_name)), nr_images_per_style_random)
  images_labels.extend([(image, folder_name) for image in random_images])

random.shuffle(images_labels)

In [ ]:
# Använd för att läsa in bilder i ordningsföljd med specifika kategorier
# Bra om man vill hålla dessa variablar statiska, när man experimenterar
# ! Om man vill köra mindre två så behöver man göra förändringar enligt första versionen

nr_of_images_per_style_first = 20
sample_styles = ['Rococo', 'Color_Field_Painting', 'Art_Nouveau_Modern', 'High_Renaissance', 'Early_Renaissance'] 

root = "/content/gdrive/MyDrive/project-art/datasets/Wikiart-dataset/wikiart/"
images_labels = []

for folder_name in sample_styles:
  images = os.listdir(os.path.join(root, folder_name))[:nr_of_images_per_style_first]
  images_labels.extend([(image, folder_name) for image in images])

random.seed(42)
random.shuffle(images_labels)

In [ ]:
print(len(images_labels))
print(images_labels)

In [ ]:
import time
import datetime

start = time.time()

image_batch = []
label_batch = [] 

for image, label in images_labels:
  image_path = os.path.join(root, label, image)
  image = cv2.imread(image_path)
  image = cv2.resize(image, (128, 128))
  image = img_to_array(image)
  image_batch.append(image)
  label_batch.append(label)

end = time.time()

print(f"It took : {datetime.timedelta(seconds=int(end - start))}")mlb = LabelBinarizer()

In [ ]:

print(len(label_batch))
print(len(image_batch))
print(set(label_batch))

In [ ]:
image_batch = np.array(image_batch, dtype="float32") / 255.0
label_batch = np.array(label_batch)
mlb = LabelBinarizer()
label_batch = mlb.fit_transform(label_batch)

In [ ]:
print([label_batch[0]])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(image_batch,label_batch,test_size=0.4,random_state=42)

In [ ]:
print(f"x_train shape : {x_train.shape}")
print(f"y_train shape : {y_train.shape}")

In [ ]:
anne = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=1e-3)
checkpoint = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)

image_data_generator = ImageDataGenerator(zoom_range = 0.2, horizontal_flip=True, shear_range=0.2)
image_data_generator.fit(x_train)

batch_size = x_train.shape[0]

history = model.fit_generator(datagen.flow(x_train, y_train, batch_size= batch_size),
               steps_per_epoch= int(np.ceil(x_train.shape[0] / batch_size)),
               epochs=30,
               verbose=2,
               callbacks=[anne, checkpoint],
               validation_data=(x_train, y_train))

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test, len(y_test))

In [ ]:
 base_model.summary()

# Learning curve

In [ ]:
def show_learning_curve():
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']

  loss = history.history['loss']
  val_loss = history.history['val_loss']

  plt.figure(figsize=(8, 8))
  plt.subplot(2, 1, 1)
  plt.plot(acc, label='Training Accuracy')
  plt.plot(val_acc, label='Validation Accuracy')
  plt.legend(loc='lower right')
  plt.ylabel('Accuracy')
  plt.ylim([min(plt.ylim()),1])
  plt.title('Training and Validation Accuracy')

  plt.subplot(2, 1, 2)
  plt.plot(loss, label='Training Loss')
  plt.plot(val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.ylabel('Cross Entropy')
  plt.ylim([0,1.0])
  plt.title('Training and Validation Loss')
  plt.xlabel('epoch')
  plt.show()

In [ ]:
show_learning_curve()

In [ ]:
base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in model_d.layers[:fine_tune_at]:
  layer.trainable =  False

Number of layers in the base model:  427


In [ ]:
model.summary()

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test, len(y_test))

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer = RMSprop(lr=1e-3/10), loss='categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
show_learning_curve()

#Bra info såsom commands etc

commands
--------
to see gpu usage : watch nvidia-smi (in terminal)

För att effektivisera:
köra parallellt med det egna grafikkortet också
https://colab.research.google.com/github/d2l-ai/d2l-en-colab/blob/master/chapter_computational-performance/auto-parallelism.ipynb#scrollTo=SIRBUh2fAGcg

#Visual result 

In [ ]:
# None